In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [33]:
spark = SparkSession \
    .builder \
    .appName("readfromcsv") \
    .master("local[4]") \
    .getOrCreate()

22/05/05 05:21:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [41]:
schema1 = StructType([StructField("Age", IntegerType(), True),
                      StructField("Sex", StringType(), True),
                      StructField("BP", StringType(), True),
                      StructField("Cholesterol", StringType(), True),
                      StructField("Na_to_K", IntegerType(), True),
                      StructField("Drug", StringType(), True)
                     ])

In [42]:
customer = spark.readStream.format("csv").schema(schema1)\
    .option("header", True).option("maxFilePerTrigger", 1)\
    .load(r"../data/streaming")

In [43]:
customer.isStreaming

True

In [44]:
customer.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- Cholesterol: string (nullable = true)
 |-- Na_to_K: integer (nullable = true)
 |-- Drug: string (nullable = true)



In [45]:
average_age = customer.groupBy("Sex")\
    .agg((avg("Age").alias("average_age")), (count("Sex").alias("count")))\
    .sort(desc("average_age"))

In [46]:
query = average_age.writeStream.format("console").outputMode("complete").start()

22/05/05 05:27:56 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8b142dd8-a42a-4bdc-bb2c-1c279cc564d6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----------------+-----+
|Sex|      average_age|count|
+---+-----------------+-----+
|  M|45.93269230769231|  104|
|  F|          42.5625|   96|
+---+-----------------+-----+

